# MC Analysis Toolkits

## 1. Setup Re-opt for lower energy conformers

In [1]:
import pandas as pd
import numpy as np
import warnings
import os
import pathlib

FOLDER = '2PhMCP_pTSA_MC_xtb2'
STRUCT_PREFIX = '2PhMCP_pTSA_xtb2'
NEXT_FOLDER = '2PhMCP_pTSA_postMC_def2tzvp'

def extract_energy(folder:str,struct_folder_prefix:str):
    energy = []
    energy_index = []
    for i in range(0,100):
        with open(f"../{folder}/{struct_folder_prefix}_{i}/{struct_folder_prefix}_{i}.out",'r') as f_mc:
            lines = f_mc.readlines()
            for line in lines:
                if "The optimization did not converge but reached the maximum" in line:
                    break
                if "Final Gibbs free energy" in line:
                    energy.append(float(line.split()[-2]))
                    energy_index.append(i)
    return energy_index, energy

def gen_inp_qsub(filename_prefix, init_structure:str, folder:str):
    '''
    FILE_STRUCTURE
    --------------
    ACS_Group (or whatever else that you wanna name it)
        |___Notebooks (where this ipynb is in)
        |___folder (PARAMETER)
            |___filename_prefix (PARAMETER)
                |___filename_prefix.sh
                |___filename_prefix.inp
                |___init_structure (PARAMETER).xyz 
    '''
    ORCA_Template = """!B3LYP D3BJ def2-TZVP OPT FREQ CPCM(2,2,2-trifluoroethanol) tightSCF RIJCOSX miniprint

%maxcore 4000
%pal
nproc 16
end

*xyzfile 0 1 {INIT_STRUCTURE}

"""
    qsub_Template = """#!/bin/bash
#PBS -l select=1:ncpus=16:mpiprocs=16:mem=64gb
#PBS -l walltime=24:00:00
#PBS -N {STRUCT_FOLDERNAME}

module load OpenMPI/4.1.6-GCC-13.2.0
module load ORCA/6.1.0-gompi-2023b-avx2

cd /rds/general/user/ah1123/home/ACS_Group/{FOLDER_NAME}/{STRUCT_FOLDERNAME}

/sw-eb/software/ORCA/6.1.0-gompi-2023b-avx2/bin/orca {FILENAME_INPUT} > {FILENAME_OUTPUT}

"""
    orca_data = {"INIT_STRUCTURE": f"{init_structure}.xyz"}
    orca_content = ORCA_Template.format(**orca_data)
    qsub_data = {"FILENAME_INPUT": f"{filename_prefix}.inp",
                "FILENAME_OUTPUT": f"{filename_prefix}.out",
                "STRUCT_FOLDERNAME":f"{filename_prefix}",
                "FOLDER_NAME": f"{folder}"}
    qsub_content = qsub_Template.format(**qsub_data)
    with open(f"../{folder}/{filename_prefix}/{filename_prefix}.inp","w") as f_orca:
        f_orca.write(orca_content)
    with open(f"../{folder}/{filename_prefix}/{filename_prefix}.sh","w") as f_qsub:
        f_qsub.write(qsub_content)

In [13]:
index, energy = zip(extract_energy(folder = FOLDER, struct_folder_prefix=STRUCT_PREFIX))
zipped_energy = zip(index[0],energy[0])
sorted_energy = sorted(zipped_energy, key = lambda x:x[1])

In [14]:
sorted_energy

[(94, -87.05696747),
 (64, -87.0563638),
 (53, -87.05634367),
 (79, -87.05571804),
 (46, -87.05536812),
 (68, -87.05535419),
 (0, -87.05500471),
 (81, -87.0542593),
 (35, -87.05411329),
 (85, -87.05401557),
 (39, -87.05390754),
 (66, -87.0536028),
 (67, -87.0533736),
 (78, -87.05316604),
 (6, -87.05315108),
 (73, -87.05306269),
 (31, -87.05282162),
 (86, -87.05274938),
 (57, -87.05245244),
 (2, -87.05225335),
 (24, -87.05205011),
 (74, -87.05193846),
 (5, -87.05171554),
 (10, -87.05170379),
 (51, -87.05162914),
 (41, -87.05152241),
 (91, -87.05137837),
 (29, -87.05098562),
 (75, -87.05078805),
 (30, -87.05064897),
 (76, -87.05047407),
 (61, -87.05046469),
 (43, -87.05042081),
 (95, -87.05036341),
 (92, -87.05028829),
 (36, -87.04988276),
 (42, -87.04982454),
 (58, -87.04965334),
 (27, -87.04964343),
 (1, -87.04961013),
 (20, -87.04926712),
 (84, -87.04923481),
 (54, -87.0486584),
 (89, -87.04864101),
 (98, -87.0486357),
 (96, -87.04858173),
 (19, -87.04854361),
 (15, -87.04824356),
 (3

In [3]:
if not os.path.isdir(f"../{NEXT_FOLDER}"):
    os.mkdir(f"../{NEXT_FOLDER}")

for i,packed_energy in enumerate(sorted_energy[:25]):
    if not os.path.isdir(f"../{NEXT_FOLDER}/{NEXT_FOLDER}_{i}"):
        os.mkdir(f"../{NEXT_FOLDER}/{NEXT_FOLDER}_{i}")
    os.system(f"cp ../{FOLDER}/{STRUCT_PREFIX}_{packed_energy[0]}/{STRUCT_PREFIX}_{packed_energy[0]}.xyz ../{NEXT_FOLDER}/{NEXT_FOLDER}_{i}/{STRUCT_PREFIX}_{packed_energy[0]}.xyz")
    gen_inp_qsub(filename_prefix=f"{NEXT_FOLDER}_{i}",
                 init_structure=f"{STRUCT_PREFIX}_{packed_energy[0]}",
                 folder=NEXT_FOLDER)

## Process Re-Opt

In [2]:
reopt_energy_res = []
reopt_index = []
for i in range(0,25):
    j = 0
    valid_flag = []
    while True:
        valid_flag.append(True)
        try:
            with open(f"../{NEXT_FOLDER}/{NEXT_FOLDER}_{i}/{NEXT_FOLDER}_{i}" + j * "_re" + ".out","r") as reopt_out:
                lines = reopt_out.readlines()
                for line in lines:
                    if "The optimization did not converge but reached the maximum" in line:
                        valid_flag[-1] = False
                    if "***imaginary mode***" in line:
                        valid_flag[-1] = False
                    if "Final Gibbs free energy" in line:
                        reopt_energy_res.append(float(line.split()[-2]))
                        reopt_index.append(i)
            j += 1
        except:
            break
    if not valid_flag[-2]:
        warnings.warn(f"Structure No. {i} has imaginary frequency / has not yet converged!")

/var/folders/wc/zk8pc51n13d38w8ylrcqnr440000gn/T/ipykernel_15412/3107909821.py:23: UserWarning: Structure No. 1 has imaginary frequency / has not yet converged!
  warnings.warn(f"Structure No. {i} has imaginary frequency / has not yet converged!")
/var/folders/wc/zk8pc51n13d38w8ylrcqnr440000gn/T/ipykernel_15412/3107909821.py:23: UserWarning: Structure No. 11 has imaginary frequency / has not yet converged!
  warnings.warn(f"Structure No. {i} has imaginary frequency / has not yet converged!")
/var/folders/wc/zk8pc51n13d38w8ylrcqnr440000gn/T/ipykernel_15412/3107909821.py:23: UserWarning: Structure No. 13 has imaginary frequency / has not yet converged!
  warnings.warn(f"Structure No. {i} has imaginary frequency / has not yet converged!")
/var/folders/wc/zk8pc51n13d38w8ylrcqnr440000gn/T/ipykernel_15412/3107909821.py:23: UserWarning: Structure No. 14 has imaginary frequency / has not yet converged!
  warnings.warn(f"Structure No. {i} has imaginary frequency / has not yet converged!")
/var/

In [3]:
f"../{NEXT_FOLDER}/{NEXT_FOLDER}_{i}/{NEXT_FOLDER}_{i}" + j * "_re" + ".out"

NameError: name 'NEXT_FOLDER' is not defined

In [24]:
reopt_energy_res

[-1668.8817794,
 -1668.8772384,
 -1668.88047111,
 -1668.881146,
 -1668.8787421,
 -1668.87834503,
 -1668.88038313,
 -1668.87727808,
 -1668.87923029,
 -1668.87793868,
 -1668.87852014,
 -1668.87683959,
 -1668.87806248,
 -1668.87349382,
 -1668.87520546,
 -1668.8790707,
 -1668.87532014,
 -1668.87739625,
 -1668.87392195,
 -1668.87310751,
 -1668.87453141,
 -1668.87455497]

In [3]:
re_zip = zip(reopt_index,reopt_energy_res)
re_sort = sorted(re_zip, key= lambda x:x[1])

In [4]:
re_sort

[(0, -1668.8817794),
 (3, -1668.881146),
 (2, -1668.88047111),
 (6, -1668.88038313),
 (8, -1668.87923029),
 (16, -1668.8790707),
 (4, -1668.8787421),
 (10, -1668.87852014),
 (5, -1668.87834503),
 (12, -1668.87806248),
 (9, -1668.87793868),
 (18, -1668.87739625),
 (7, -1668.87727808),
 (1, -1668.8772384),
 (11, -1668.87683959),
 (17, -1668.87532014),
 (15, -1668.87520546),
 (23, -1668.87455497),
 (22, -1668.87453141),
 (20, -1668.87392195),
 (13, -1668.87349382),
 (21, -1668.87310751)]

In [5]:
len(re_sort)

22